<a href="https://colab.research.google.com/github/LuchoGabba/qibo/blob/main/Application_Colab.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>

# Talk from Stefano Carrazza

Quantum simulation with a hardware acceleration (arXiv:2009.01845)

https://www.youtube.com/watch?v=Aoc4nLmplTE

In [6]:
!nvidia-smi

Sat Oct 28 23:12:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install qibo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 10.7 MB/s eta 0:00:00


In [4]:
import qibo

In [5]:
qibo.__version__

'0.2.1'

In [7]:
%%time
from qibo.models import QFT
c = QFT(25)
c()

[Qibo 0.2.1|INFO|2023-10-28 23:12:41]: Using tensorflow backend on /device:GPU:0
INFO:qibo.config:Using tensorflow backend on /device:GPU:0


CPU times: user 8 s, sys: 2.31 s, total: 10.3 s
Wall time: 44.2 s


(0.00017+0j)|0000000000000000000000000> + (0.00017+0j)|0000000000000000000000001> + (0.00017+0j)|0000000000000000000000010> + (0.00017+0j)|0000000000000000000000011> + (0.00017+0j)|0000000000000000000000100> + (0.00017+0j)|0000000000000000000000101> + (0.00017+0j)|0000000000000000000000110> + (0.00017+0j)|0000000000000000000000111> + (0.00017+0j)|0000000000000000000001000> + (0.00017+0j)|0000000000000000000001001> + (0.00017+0j)|0000000000000000000001010> + (0.00017+0j)|0000000000000000000001011> + (0.00017+0j)|0000000000000000000001100> + (0.00017+0j)|0000000000000000000001101> + (0.00017+0j)|0000000000000000000001110> + (0.00017+0j)|0000000000000000000001111> + (0.00017+0j)|0000000000000000000010000> + (0.00017+0j)|0000000000000000000010001> + (0.00017+0j)|0000000000000000000010010> + (0.00017+0j)|0000000000000000000010011> + ...

We can change to GPU:
* Runtime/Entorno de ejecución
* Change runtime type/Cambiar tipo de entorno de ejecución
* Select T4 GPU

It's pretty faster than in CPU.

In [8]:
c.final_state

(0.00017+0j)|0000000000000000000000000> + (0.00017+0j)|0000000000000000000000001> + (0.00017+0j)|0000000000000000000000010> + (0.00017+0j)|0000000000000000000000011> + (0.00017+0j)|0000000000000000000000100> + (0.00017+0j)|0000000000000000000000101> + (0.00017+0j)|0000000000000000000000110> + (0.00017+0j)|0000000000000000000000111> + (0.00017+0j)|0000000000000000000001000> + (0.00017+0j)|0000000000000000000001001> + (0.00017+0j)|0000000000000000000001010> + (0.00017+0j)|0000000000000000000001011> + (0.00017+0j)|0000000000000000000001100> + (0.00017+0j)|0000000000000000000001101> + (0.00017+0j)|0000000000000000000001110> + (0.00017+0j)|0000000000000000000001111> + (0.00017+0j)|0000000000000000000010000> + (0.00017+0j)|0000000000000000000010001> + (0.00017+0j)|0000000000000000000010010> + (0.00017+0j)|0000000000000000000010011> + ...

In [10]:
print(c.draw())

q0 :     ─H─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U1─U ...
q1 :     ───o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q2 :     ──────o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q3 :     ─────────o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q4 :     ────────────o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q5 :     ───────────────o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q6 :     ──────────────────o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q7 :     ─────────────────────o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q8 :     ────────────────────────o──|──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q9 :     ───────────────────────────o──|──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q10:     ──────────────────────────────o──|──|──|──|──|──|──|──|──|──|──|──|──| ...
q11:     ─────────────────────────────────o──|──|──|──|──|──|──|──|──|──|──|